In [191]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [192]:
def scrape_coches_ocasion(url, diccionario):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
            print(f"El motivo REASON por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
            print(f"El motivo TEXT por el que la llamada falló es {llamada.text}")  # Imprime la razón de la falla.
            return None
    else:
        soup = BeautifulSoup(llamada.text, 'html.parser')
        coches_precios = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"}) # Sopa de coches y precios originales
        precios_descuento = soup.find_all('span', {"class":"cardVehicle_finance__SG6JV"}) # Sopa de precios con descuento
        km = soup.find_all('span', {"class":"characteristics_elements__Mb1S_"}) # Sopa de km

        for kilom in km: # Metiendo los Km en el diccionario
            if kilom.get_text('data-test').endswith('Km'):
                diccionario['km'].append(kilom.get_text('data-test'))

        for desc in precios_descuento: # Metiendo los precios con descuento en el diccionario
            if "€" in desc.text:
                diccionario['precio_rebajado'].append(desc.text)

        for nombre in coches_precios: # Metiendo los coches y precios originales en el diccionario
            if "€" in nombre.text:
                diccionario['precio_original'].append(nombre.text)
            else:
                diccionario['coche'].append(nombre.text)

        # Verificar que todas las listas tengan el mismo len
        if len(diccionario['coche']) == len(diccionario['precio_original']) == len(diccionario['precio_rebajado']) == len(diccionario['km']):
            pass
        else: # Si no tienen el mismo len, buscar el problema y corregirlo
            counter = 0
            for i in coches_precios:
                counter += 1
                if counter %2 == 0 and "€" in i.text:
                    diccionario['precio_original'].insert(counter-1, diccionario['precio_rebajado'][counter-1])
                    print(f"El indice es: {counter-1}")
                else:
                    pass
        
        return pd.DataFrame(diccionario)


url = "https://www.ocasionplus.com/coches-ocasion?v2&orderBy=morePopular"
coches = {'coche':[], 'precio_rebajado':[], 'precio_original':[], 'km':[]} # Crea diccionario para almacenar todos los datos del DataFrame
df = scrape_coches_ocasion(url, coches)


El indice es: 1


In [193]:
len(coches['coche']) == len(coches['precio_rebajado']) == len(coches['precio_original']) == len(coches['km'])

True

In [194]:
print(len(coches['coche']))
print(len(coches['precio_rebajado']))
print(len(coches['precio_original']))
print(len(coches['km']))

20
20
20
20


In [199]:
df

,coche,precio_rebajado,precio_original,km
0,Toyota GR Yaris,34.500€,37.950€,27.241 Km
1,Mini MINI,16.200€,16.200€,80.000 Km
2,Seat León,18.536€,20.390€,94.404 Km
3,Peugeot Rifter,17.182€,19.800€,70.731 Km
4,Fiat Punto,5.450€,4.550€,172.442 Km
5,Volkswagen Golf,28.772€,31.650€,64.393 Km
6,KIA XCeed,14.500€,15.900€,46.353 Km
7,MG ZS,18.536€,20.390€,10.423 Km
8,Hyundai i30,16.536€,18.190€,78.072 Km
9,Mitsubishi Eclipse Cross,24.818€,27.300€,18.531 Km
